In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        continue
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 랜덤 시드 고정
import random
import librosa
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

seed = 42

random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
# 학습할 음악들의 장르 정답 csv 파일
train_info_csv = pd.read_csv('/kaggle/input/2023-ml-project4/train_labels.csv')

# 제출에 사용할 csv 파일
submit = pd.read_csv('/kaggle/input/2023-ml-project4/submit.csv')
test_info_csv = submit['id']

In [7]:
# label
train_info_csv['genre'].unique()

array(['rock', 'country', 'metal', 'hiphop', 'pop', 'classical', 'disco',
       'reggae', 'blues', 'jazz'], dtype=object)

In [8]:
def extract_rhythm_features(file_path):
    # 반환할 feature list 선언
    feature = []
    y, sr = librosa.load(file_path, sr=22050)
    onset_envelope = librosa.onset.onset_strength(y=y, sr=sr)
    
    # autocorrelation tempogram
    tempogram = librosa.feature.tempogram(onset_envelope=onset_envelope, sr=sr)  # (384, 1293)
    #print(tempogram.shape)
    tempogram_feature = np.abs(np.mean(tempogram, axis=1))   # (384, 1293) -> (384, ), 복소수를 없애기 위해 절대값 처
    #print(tempogram_feature.shape)
    
    feature = tempogram_feature
    
    return feature

In [9]:
def extract_spectral_features(file_path):
    # 반환할 feature list 선언
    feature = []
    y, sr = librosa.load(file_path, sr=22050)

    spectrogram = np.abs(librosa.stft(y))
    power_spectrogram = spectrogram ** 2
    melspectrogram = librosa.feature.melspectrogram(S=power_spectrogram)
    melspectrogram_db = librosa.power_to_db(melspectrogram)
    
    # chromagram
    chromagram = librosa.feature.chroma_stft(S=power_spectrogram) # (12, 1293)
    chromagram_feature = np.mean(chromagram, axis=1)  # (12, 1293) -> (12, )
     
    # mfcc
    mfcc = librosa.feature.mfcc(S=melspectrogram_db)   # (20, 1293)
    mfcc_feature = np.mean(mfcc, axis=1) # (20, )
    
    feature = np.concatenate((chromagram_feature, mfcc_feature))

    return feature

In [10]:
def feature_loader(data_info, split=None, rootpath=None, domain=None):
    split = split.upper()
    info_dict = {}
    
    if split=='TRAIN':
        
            
        return info_dict
        
    elif split=='TEST':
        
            
        return info_dict
    
    else:
        raise Exception("Check split")